A model on $n$ vertices with order $d$.
$K$ vertices forming a clique, meaning that any size-d tuple within the clique is connected by a hyperedge.
All other size-$d$ tuples form a hyperedge with probability $q = 1/2$ .

In [295]:
import math
import time as tm
from itertools import permutations

import numpy as np

In [296]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g

In [297]:
##create a graph with planted clique
def generate_graph(num_vertices, pr, clq_size):
    def create_edges():
        for i in range(num_vertices):
            for j in range(i + 1, num_vertices):
                for k in range(j + 1, num_vertices):
                    if np.random.uniform(0, 1, 1) < pr: # probability of creating edges
                        vec[[i, j, k]] += 1
                        add_all(i, j, k, g)

    def plant_clique():
        for i in range(clq_size):
            for j in range(i + 1, clq_size):
                for k in range(j + 1, clq_size):
                    vec[[clq_vertex[i], clq_vertex[j], clq_vertex[k]]] += 1
                    add_all(clq_vertex[i], clq_vertex[j], clq_vertex[k], g)

    g = np.zeros((num_vertices, num_vertices, num_vertices)) # graph on n vertices
    vec = np.zeros(num_vertices, ) # edge
    create_edges()
    clq_vertex = np.random.choice(range(num_vertices), clq_size, replace=False) # randomly choose a given number of clique vertices
    plant_clique() # plant a clique to vertices (add edges between every two pairs of vertices)

    return g, vec, clq_vertex

In [298]:
# calculate total number of edges
def count_edge(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [299]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    active_count = np.count_nonzero(vec)  # number of vertices that are associated with at least 1 edge
    edge_sum = count_edge(g, num_vertices)  # number of edges
    return edge_sum == math.comb(active_count, 3)

In [300]:
# remove edge
def remove_edges(g, num_vertices, vec, curr_idx):
    vec[curr_idx] = 0
    for j in range(0, num_vertices):
        for k in range(j+1, num_vertices):
            if g[curr_idx, j, k] == 1:
                vec[[j, k]] -= 1
                remove_all(curr_idx, j, k, g)
    return g, vec

### Generate a graph

In [301]:
N = 100  # number of vertices
P = 0.5  # probability of an edge being included
K = 10  # clique size

start_generate = tm.time()
res = generate_graph(N, P, K)
G, V, clique_vertices = res[0], res[1], res[-1] # G: graph V: vector storing the number of edges associated with each vertex
G_0 = G.copy()

# print(f"edge-occurrence vector: {V}\n")
# print(f"set of clique vertices after generation phase: {clique_vertices}\n")

time_generate = np.round(tm.time() - start_generate, 3)
print(f"Time taken to generate graph: {time_generate} seconds")

Time taken to generate graph: 0.48 seconds


### Removal Phase

In [302]:
start_removal = tm.time()
itr = 0
removed = []  # keep track of the vertices removed from the original graph to form a clique

while not is_clique(G, V, N):
    itr += 1
    curr = -1
    idx_sorted = np.argsort(V)

    for idx in range(N):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break

    A = remove_edges(G, N, V, curr)
    
    G, V = A[0], A[1]

print(f"is a clique at iteration #{itr}!!!")
print(f"Time taken to execute removal: {np.round(tm.time() - start_removal, 3)} seconds")

is a clique at iteration #90!!!
Time taken to execute removal: 2.417 seconds


In [303]:
# print(f"number of iterations in the removal phase: {itr}")

In [304]:
included = np.setdiff1d(np.arange(N), removed)
included

array([ 5,  8, 19, 43, 57, 60, 71, 83, 94, 98])

### Inclusion Phase

In [305]:
def inclusion_phase(target, in_set, g):
    def connected(): 
        for j in range(len(in_set)):
            for k in range(j + 1, len(in_set)):
                if g[target_idx, in_set[j], in_set[k]] != 1:
                    return False
        return True

    for target_idx in target:
        if connected():
            print(f"add {target_idx} to clique!")
            in_set = np.append(in_set, target_idx)
            

    return in_set

In [306]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
print(f"Time taken to execute inclusion: {tm.time() - start_include} seconds")


check 14

check 13

check 44

check 95

check 77

check 42

check 32

check 49

check 54

check 31

check 53

check 62

check 23

check 3

check 37

check 64

check 65

check 85

check 25

check 56

check 47

check 84

check 16

check 69

check 41

check 33

check 1

check 96

check 63

check 4

check 34

check 72

check 7

check 67

check 29

check 76

check 2

check 6

check 75

check 38

check 90

check 27

check 89

check 10

check 81

check 9

check 45

check 79

check 93

check 58

check 59

check 61

check 70

check 22

check 50

check 36

check 68

check 51

check 92

check 88

check 11

check 48

check 91

check 12

check 39

check 28

check 74

check 78

check 86

check 73

check 35

check 26

check 99

check 24

check 66

check 18

check 0

check 97

check 21

check 55

check 17

check 46

check 87

check 82

check 52

check 40

check 80

check 20

check 15

check 30
Time taken to execute inclusion: 0.00023102760314941406 seconds


In [307]:
print(f"Set of clique vertices after removal phase: {np.sort(included)}\n")
print(f"Set of clique vertices after inclusion phase: {np.sort(res)}")

Set of clique vertices after removal phase: [ 5  8 19 43 57 60 71 83 94 98]

Set of clique vertices after inclusion phase: [ 5  8 19 43 57 60 71 83 94 98]
